In [1]:
# Warning control
import warnings
warnings.filterwarnings("ignore")


In [2]:
from crewai import Agent, Task, Crew


In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"


In [4]:
support_agent = Agent(
    role="Senior Support Representative",
    goal="Be the most friendly and helpful support representative in your team.",
    backstory=(
        "You work at crewAI (https://crewai.com) and are now working on providing "
        "support to {customer}, a super important customer for your company. "
        "You need to make sure that you provide the best support! Make sure to "
        "provide full complete answers, and make no assumptions."
    ),
    allow_delegation=False,
    verbose=True,
)


In [5]:
support_quality_assurance_agent = Agent(
    role="Support Quality Assurance Specialist",
    goal="Get recognition for providing the best support qualituy assurance in your team.",
    backstory=(
        "Your work at crewAI (https://crewai.com) and are now working with yort team on a "
        "request from {customer} ensuring that the support representative is providing the "
        "best support possible. \nYou need to make sure that the support representative " 
        "is providing full complete answers, and make no assumptions."
    ),
    verbose=True,
)


In [6]:
# SerperDevTool -> external service that allows the agent to do search on google
# ScrapeWebsiteTool -> single scrapper that allows the agent to access a given URL and extract contents of that
# WebsiteSearchTool -> RAG over a website ( semantic search over the content of that website )
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool


In [7]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/concepts/crews",
)


In [8]:
inquiry_resolytuon = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
        "{inquiry}\n\n"
        "{person} from {customer} if the one that reached out. Make sure to use everything "
        "your know to provide the best support possible. Your must strive to provide a "
        "complete and accurate response to the customer's inquiry."
    ),
    expected_output=(
        "A detailed, informative response to the customer's inquiry that address "
        "all aspects of their question.\n The response should include references "
        "to everything you used to find the answer, including external data or "
        "solutions. Ensure the answer is complete, leaving no questions unanswered, "
        "and maintain a helpful and friendly tone throughout."
    ),
    tools=[docs_scrape_tool], # using interpolation
    agent=support_agent,
)


In [9]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


In [10]:
crew = Crew(
    agents=[support_agent, support_quality_assurance_agent],
    tasks=[inquiry_resolytuon, quality_assurance_review],
    verbose=2,
    memory=True, # enables short-tem, long-term, and the entity memories
)


In [11]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)


 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI if the one that reached out. Make sure to use everything your know to provide the best support possible. Your must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
I need to provide detailed and informative guidance to DeepLearningAI on how to add memory to their Crew. I should use the Read website content tool to gather the necessary information.

Action: Read website content
Action Input: {} 

I encountered an error while trying to use the tool. This was the error: HTTPSConnectionPool(host='docs.crewai.com', port=443): Max retries exceeded with url: /concepts/crews (Caused by SSLError(SSLCertVerificationError(1, '[

2024-10-28 17:26:12,952 - 13351546880 - __init__.py-__init__:369 - ERROR: Exception while exporting Span batch.
Traceback (most recent call last):
  File "/opt/miniconda3/envs/agents/lib/python3.11/site-packages/urllib3/connectionpool.py", line 466, in _make_request
    self._validate_conn(conn)
  File "/opt/miniconda3/envs/agents/lib/python3.11/site-packages/urllib3/connectionpool.py", line 1095, in _validate_conn
    conn.connect()
  File "/opt/miniconda3/envs/agents/lib/python3.11/site-packages/urllib3/connection.py", line 730, in connect
    sock_and_verified = _ssl_wrap_socket_and_match_hostname(
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/agents/lib/python3.11/site-packages/urllib3/connection.py", line 909, in _ssl_wrap_socket_and_match_hostname
    ssl_sock = ssl_wrap_socket(
               ^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/agents/lib/python3.11/site-packages/urllib3/util/ssl_.py", line 469, in ssl_wrap_socket
    ssl_soc

2024-10-28 17:26:13,693 - 8711229248 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--baf466ca95d42718e5f4dd53fba5f4e2b1c35114184bc8a0ea1654ad788b79b3


Thought: I need to provide detailed and informative guidance to DeepLearningAI on how to add memory to their Crew.
Action: Read website content
Action Input: {} 

I encountered an error while trying to use the tool. This was the error: HTTPSConnectionPool(host='docs.crewai.com', port=443): Max retries exceeded with url: /concepts/crews (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)'))).
 Tool Read website content accepts these inputs: Read website content() - A tool that can be used to read https://docs.crewai.com/concepts/crews's content.



2024-10-28 17:26:15,332 - 8711229248 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--baf466ca95d42718e5f4dd53fba5f4e2b1c35114184bc8a0ea1654ad788b79b3


Thought: I need to provide detailed and informative guidance to DeepLearningAI on how to add memory to their Crew.
Action: Read website content
Action Input: {} 

I encountered an error while trying to use the tool. This was the error: HTTPSConnectionPool(host='docs.crewai.com', port=443): Max retries exceeded with url: /concepts/crews (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)'))).
 Tool Read website content accepts these inputs: Read website content() - A tool that can be used to read https://docs.crewai.com/concepts/crews's content.



2024-10-28 17:26:17,359 - 8711229248 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--baf466ca95d42718e5f4dd53fba5f4e2b1c35114184bc8a0ea1654ad788b79b3


Thought: I need to provide detailed and informative guidance to DeepLearningAI on how to add memory to their Crew.
Action: Read website content
Action Input: {} 

I encountered an error while trying to use the tool. This was the error: HTTPSConnectionPool(host='docs.crewai.com', port=443): Max retries exceeded with url: /concepts/crews (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)'))).
 Tool Read website content accepts these inputs: Read website content() - A tool that can be used to read https://docs.crewai.com/concepts/crews's content.



2024-10-28 17:26:19,703 - 8711229248 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--baf466ca95d42718e5f4dd53fba5f4e2b1c35114184bc8a0ea1654ad788b79b3


Thought: I need to provide detailed and informative guidance to DeepLearningAI on how to add memory to their Crew.
Action: Read website content
Action Input: {} 

I encountered an error while trying to use the tool. This was the error: HTTPSConnectionPool(host='docs.crewai.com', port=443): Max retries exceeded with url: /concepts/crews (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)'))).
 Tool Read website content accepts these inputs: Read website content() - A tool that can be used to read https://docs.crewai.com/concepts/crews's content.

Thought: Since I am unable to access the website content, I will provide the best guidance based on my knowledge and expertise on adding memory to a Crew for DeepLearningAI.
Final Answer: 
To add memory to your Crew for DeepLearningAI, you can follow these steps:

1. Understand the memory requirements: Determine the memory needs of Deep

In [12]:
from IPython.display import Markdown
Markdown(result)



Hi there!

Thank you for reaching out to us at DeepLearningAI with your inquiry about adding memory to your Crew. We understand the importance of memory allocation in AI models, especially those with large neural network layers or recurrent neural networks (RNNs). 

When it comes to memory requirements for AI models, it's crucial to consider the complexity and size of the model. Models with large neural network layers or RNNs typically require more memory due to the increased number of parameters and computations involved in training and inference. For instance, a convolutional neural network (CNN) used for image recognition tasks may have millions of parameters, leading to higher memory demands compared to simpler models like linear regression.

Proper memory allocation is essential for optimal AI model performance. Insufficient memory can result in out-of-memory errors or slow training and inference times as the model has to constantly switch data between memory and disk. On the other hand, having sufficient memory can accelerate the training process and enhance overall model performance.

To enhance memory usage and performance, it's vital to monitor and manage memory allocation effectively during both training and deployment stages. Techniques such as batch processing, data sharding, and model pruning can help reduce memory requirements and boost the efficiency of AI models.

In conclusion, understanding the impact of memory on AI model performance is key to making informed decisions for optimizing efficiency and effectiveness. By following the steps outlined and being mindful of memory requirements, DeepLearningAI can successfully enhance the performance of its AI models.

If you encounter any challenges or need further assistance, please don't hesitate to reach out. We're here to help!

Warm regards,

[Your Name]
Support Quality Assurance Specialist
crewAI